Importações

In [118]:

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv
from IPython.display import Audio
from scipy.io import wavfile

In [119]:
#Verificação das versões do TensorFlow e TensorFlow Hub
print('Versão do TensorFlow:', tf.__version__)
print('Versão do TensorFlow Hub:', hub.__version__)

Versão do TensorFlow: 2.13.0
Versão do TensorFlow Hub: 0.14.0


Carregar o Modelo YAMNet

In [120]:
modelo = hub.load("https://tfhub.dev/google/yamnet/1")

As classes são armazenadas na lista classes, que pode ser usada para mapear os resultados de classificação do modelo


In [121]:
classes = []
with tf.io.gfile.GFile(modelo.class_map_path().numpy()) as arquivo_csv:
  reader = csv.DictReader(arquivo_csv)
  for linha in reader:
    classes.append(linha['display_name'])

In [122]:
print("Numero de Classes : ",len(classes))
print("Classes : ",classes)

Numero de Classes :  521
Classes :  ['Speech', 'Child speech, kid speaking', 'Conversation', 'Narration, monologue', 'Babbling', 'Speech synthesizer', 'Shout', 'Bellow', 'Whoop', 'Yell', 'Children shouting', 'Screaming', 'Whispering', 'Laughter', 'Baby laughter', 'Giggle', 'Snicker', 'Belly laugh', 'Chuckle, chortle', 'Crying, sobbing', 'Baby cry, infant cry', 'Whimper', 'Wail, moan', 'Sigh', 'Singing', 'Choir', 'Yodeling', 'Chant', 'Mantra', 'Child singing', 'Synthetic singing', 'Rapping', 'Humming', 'Groan', 'Grunt', 'Whistling', 'Breathing', 'Wheeze', 'Snoring', 'Gasp', 'Pant', 'Snort', 'Cough', 'Throat clearing', 'Sneeze', 'Sniff', 'Run', 'Shuffle', 'Walk, footsteps', 'Chewing, mastication', 'Biting', 'Gargling', 'Stomach rumble', 'Burping, eructation', 'Hiccup', 'Fart', 'Hands', 'Finger snapping', 'Clapping', 'Heart sounds, heartbeat', 'Heart murmur', 'Cheering', 'Applause', 'Chatter', 'Crowd', 'Hubbub, speech noise, speech babble', 'Children playing', 'Animal', 'Domestic animals,

Download e Tratamento do Áudio

In [123]:
!curl -O https://storage.googleapis.com/audioset/yamalyzer/audio/hi-hat.wav

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  161k  100  161k    0     0   351k      0 --:--:-- --:--:-- --:--:--  350k


In [124]:
sampre_rate, audio = wavfile.read('/content/hi-hat.wav', 'rb')

In [125]:
# Reproduzir o Áudio
Audio( audio, rate= sampre_rate)

*Normalizando o áudio dividindo cada valor no array de áudio pelo valor máximo representável de um inteiro de 16 bits (32767). Essa é uma etapa comum de pré-processamento para garantir que os valores de áudio estejam na faixa de -1 a 1, o que é comum em tarefas de processamento de áudio.*

In [126]:

audio = audio / tf.int16.max
audio

array([ 0.        ,  0.        , -0.00039674, ...,  0.02240059,
        0.01290933, -0.00958281])

In [127]:
# Classificação de Áudio
scores, _, _ = modelo(audio)

In [128]:
scores

<tf.Tensor: shape=(10, 521), dtype=float32, numpy=
array([[3.67596000e-03, 1.76026821e-04, 1.91610088e-04, ...,
        3.19230661e-04, 2.20518676e-04, 8.57179657e-06],
       [1.23911304e-03, 1.55361176e-05, 1.16753972e-05, ...,
        3.98275631e-07, 1.93926780e-06, 3.63136898e-08],
       [2.47317948e-03, 6.76571741e-04, 3.16146237e-04, ...,
        6.22003747e-04, 4.21764998e-04, 1.37826892e-05],
       ...,
       [5.32582984e-04, 3.87475484e-05, 3.76790049e-05, ...,
        8.21997019e-05, 1.28249492e-04, 2.69311568e-06],
       [6.10829564e-03, 8.03872535e-05, 1.16147065e-04, ...,
        2.14011670e-05, 5.61585584e-05, 4.29277662e-07],
       [1.47255056e-03, 5.84340269e-05, 1.66142745e-05, ...,
        5.05244179e-06, 4.18727905e-06, 2.77355582e-07]], dtype=float32)>

In [129]:
#convertendo a variável scores de um tensor TensorFlow para um array NumPy para facilitar o processamento
scores = scores.numpy()
type(scores)

numpy.ndarray

In [130]:
# Determinar a Previsão com Base nas Médias das Pontuações
previsao = scores.mean(axis=0).argmax()
previsao = classes[previsao]
print("Previsão do Áudio:", previsao)

Previsão do Áudio: Music


In [131]:
# Principais Previsões do Áudio
mean_scores = np.mean(scores, axis=0)
top_n = 5
top_class_indices = np.argsort(mean_scores)[::-1][:top_n]
print("Principais Categorias no Áudio: \n")
for i in top_class_indices:
    print(classes[i])

Principais Categorias no Áudio: 

Music
Drum machine
Hands
Percussion
Drum kit
